In [11]:
pip install BeautifulSoup4


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\diebl\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [12]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\diebl\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



# scrape_billboard() scrapes the Billboard's Hot 100 and retrieves the songs name &
# artist, as well as the last week ranks, the peak rank and the weeks on chart

def scrape_billboard():
    # 1. import libraries
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    
    # 2. find url and store it in a variable
    url = "https://www.billboard.com/charts/hot-100"
    
    # 3. download html with a get request
    response = requests.get(url)
    if response.status_code != 200:
        print("Scraping failed. Status code " + str(response.status_code))
    
    # 4. parse html (create the 'soup')
    soup = BeautifulSoup(response.content, "html.parser")
    soup.select("#main > div > span > div > div > div.lister > table > tbody > tr:nth-child(1) > td.titleColumn > a")

    # 5. retrieve/extract the desired info
    # song titles
    songs = soup.find_all("span", class_="chart-element__information__song text--truncate color--primary")
    # artists
    artists = soup.find_all("span", class_="chart-element__information__artist text--truncate color--secondary")
    # last week
    last_week_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--last")
    # peak rank
    peak_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--peak")
    # weeks on chart
    weeks_on_chart = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--week")

    for i in [songs, artists, last_week_ranks, peak_ranks, weeks_on_chart]:
        for j in range(len(i)):
            i[j] = i[j].getText()
        
    billboard = pd.DataFrame(
        {"song": songs,
         "artist": artists,
         "last_week_rank": last_week_ranks,
         "peak_rank": peak_ranks,
         "weeks_on_chart": weeks_on_chart})
    return billboard

In [36]:
import numpy as np

In [1]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
    
    # 2. find url and store it in a variable
url = "https://www.billboard.com/charts/hot-100"
    
    # 3. download html with a get request
response = requests.get(url)
if response.status_code != 200:
    print("Scraping failed. Status code " + str(response.status_code))
    
    # 4. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
soup.select("#main > div > span > div > div > div.lister > table > tbody > tr:nth-child(1) > td.titleColumn > a")

    # 5. retrieve/extract the desired info
    # song titles
songs = soup.find_all("span", class_="chart-element__information__song text--truncate color--primary")
    # artists
artists = soup.find_all("span", class_="chart-element__information__artist text--truncate color--secondary")
    # last week
last_week_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--last")
    # peak rank
peak_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--peak")
    # weeks on chart
weeks_on_chart = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--week")

for i in [songs, artists, last_week_ranks, peak_ranks, weeks_on_chart]:
    for j in range(len(i)):
        i[j] = i[j].getText()
        
billboard = pd.DataFrame(
        {"song": songs,
         "artist": artists,
         "last_week_rank": last_week_ranks,
         "peak_rank": peak_ranks,
         "weeks_on_chart": weeks_on_chart})
billboard

,song,artist,last_week_rank,peak_rank,weeks_on_chart
0,Stay,The Kid LAROI & Justin Bieber,2,1,13
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11
2,Fancy Like,Walker Hayes,5,3,16
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5
4,Bad Habits,Ed Sheeran,6,2,15
...,...,...,...,...,...
95,Esta Danada,Ivan Cornejo,-,96,1
96,Whiskey And Rain,Michael Ray,-,97,1
97,TSU,Drake,84,9,5
98,Love All,Drake Featuring JAY-Z,87,10,5


In [57]:
user_song = input("What song do you like? ")

What song do you like? My Universe


def song_box():
    song = input("What song do you like? ")
    return song


song_box()

In [51]:
song = billboard.loc[billboard['song'] == user_song]
song

,song,artist,last_week_rank,peak_rank,weeks_on_chart
99,Moon,Kanye West,76,17,5


In [59]:
for chanson in billboard['song']:
    if chanson == user_song:
        print(billboard['song'].iloc[+1])
    else:
        print('Your favorite song is not in the chart.')
    

Stay
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the chart.
Your favorite song is not in the ch

In [37]:
user_song = input("What song do you like? ")

What song do you like? Moon


In [ ]:
# solution given by Abhi

pos = -1
for index, song in enumerate(billboard['song'].tolist()):
    if song == user_song:
        pos = index

if pos >=0:
    if pos == len(billboard)-1:
        print("no next song in the list")
    else :
        print(billboard['song'].iloc[pos+1])
else:
    print('no match found')

In [ ]:
def hot_recommender():
    # import scraping function & random int generator
    from scrape_billboard import scrape_billboard
    from random import randint
    
    # scrape billboard hot 100
    billboard = scrape_billboard()
    
    # ask for song
    song = input("What song do you like? ")
    
    # check if song is on billboard hot 100
    song_row = billboard[billboard["song"].str.contains(song)]
    if len(song_row) == 0:
        print("Your song is not hot")
    else:
        check_song = input("Did you mean " + song_row["song"].values[0] + " by " + song_row["artist"].values[0] + "? ")    
    
        # if song is in billboard hot 100, recommend another random hot song
        if check_song == "yes":
            print("That's a hot song.")
            random_song = randint(0, len(billboard)-1)
            print("You might also like " + billboard["song"][random_song] + " by " + billboard["artist"][random_song])
        else:
            print("Ah, not the one I had in mind.")

In [2]:
!pip install spotipy

You should consider upgrading via the 'c:\users\diebl\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="af3a4e21d9974f798b0ddef081728f2b",
                                                           client_secret="99a65d20eff04d64bcf24b11824dffc4"))

results = sp.search(q='artist: Queen', limit=2)
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A+Queen&type=track&offset=0&limit=2',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1dfeR4HaWDbWqFHLkxsg1d'},
       'href': 'https://api.spotify.com/v1/artists/1dfeR4HaWDbWqFHLkxsg1d',
       'id': '1dfeR4HaWDbWqFHLkxsg1d',
       'name': 'Queen',
       'type': 'artist',
       'uri': 'spotify:artist:1dfeR4HaWDbWqFHLkxsg1d'}],
     'available_markets': ['CA', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6X9k3hSsvQck2OfKYdBbXr'},
     'href': 'https://api.spotify.com/v1/albums/6X9k3hSsvQck2OfKYdBbXr',
     'id': '6X9k3hSsvQck2OfKYdBbXr',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273ce4f1737bc8a646c8c4bd25a',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02ce4f1737bc8a646c8c4bd25a',
       'width': 300},
      {'height':

In [5]:
tracks =billboard['song'].tolist()
tracks

['Stay',
 'Industry Baby',
 'Fancy Like',
 'Way 2 Sexy',
 'Bad Habits',
 'Good 4 U',
 'Kiss Me More',
 'Levitating',
 'Knife Talk',
 'Essence',
 'Shivers',
 'My Universe',
 'Need To Know',
 'Save Your Tears',
 'Heat Waves',
 "Beggin'",
 'Montero (Call Me By Your Name)',
 'You Right',
 "If I Didn't Love You",
 'Girls Want Girls',
 'Traitor',
 'Take My Breath',
 'Sharing Locations',
 'Chasing After You',
 'Pepas',
 'Thats What I Want',
 'Wockesha',
 'Cold Beer Calling My Name',
 'Happier Than Ever',
 'Fair Trade',
 'Deja Vu',
 'Cold Heart (PNAU Remix)',
 "Memory I Don't Mess With",
 'A-O-K',
 'Love Nwantiti (Ah Ah Ah)',
 'Intro (Hate On Me)',
 'Leave The Door Open',
 'Gyalis',
 'Peaches',
 'Leave Before You Love Me',
 'Love Again',
 'My Boy',
 'Buy Dirt',
 'Hurricane',
 'I Was On A Boat That Day',
 'Family Ties',
 'Meet Me At Our Spot',
 'Wild Side',
 'Late At Night',
 'Have Mercy',
 'You Should Probably Leave',
 'Whole Lotta Money',
 'Cold As You',
 'You Time',
 "Drunk (And I Don't Wann

In [6]:
track_ids = []
for track in tracks:
    results = sp.search(q='track: {0}'.format(track), limit=1)
    track_ids.append(results['tracks']['items'][0]['id'])

In [7]:
track_ids

['5HCyWlXZPP0y6Gqq8TgA20',
 '27NovPIUIRrOZoCHxABJwK',
 '3ZozFqJJlDBNIm4xqxn2ZD',
 '0k1WUmIRnG3xU6fvvDVfRG',
 '6PQ88X9TkUIAUIZJHW2upE',
 '4ZtFanR9U6ndgddUvNcjcG',
 '748mdHapucXQri7IAO8yFK',
 '463CkQjx2Zk1yXoBuierM9',
 '2BcMwX1MPV6ZHP4tUT9uq6',
 '5FG7Tl93LdH117jEKYl3Cm',
 '75MNhvTCCKsST3YqqUiU9r',
 '3FeVmId7tL5YN8B7R3imoM',
 '3Vi5XqYrmQgOYBajMWSvCi',
 '37BZB0z9T8Xu7U3e65qxFy',
 '3USxtqRwSYz57Ewm6wWRMp',
 '3Wrjm47oTz2sjIgck11l5e',
 '1SC5rEoYDGUK4NfG82494W',
 '0k4d5YPDr1r7FX77VdqWez',
 '5f3MXmTmstozFg0BH1yPUk',
 '37Nqx7iavZpotJSDXZWbJ3',
 '5CZ40GBx1sQ9agT82CLQCT',
 '6OGogr19zPTM4BALXuMQpF',
 '3J4VKkUpLvomXJbn9ZAypO',
 '1ZIKpMTRPqhwAoVXWY3tXI',
 '5fwSHlTEWpluwOM0Sxnh5k',
 '0e8nrvls4Qqv5Rfa2UhqmO',
 '0nqhKXDjsyBMvbeWmgijD0',
 '3dV1fjKOlLnaNK5MeZHXGv',
 '4RVwu0g32PAqgUiJoXsdF8',
 '40iJIUlhi6renaREYGeIDS',
 '6HU7h9RYOaPRFeh0R3UeAr',
 '6zSpb8dQRaw0M1dK8PBwQz',
 '7BQE1P0NUalHSvFtrxasx9',
 '1sx0XsWUusUoPstbvokZjP',
 '2Xr1dTzJee307rmrkt8c0g',
 '4s7TIubkdsgdtDEDFEAYVL',
 '7MAibcTli4IisCtbHKrGMh',
 

In [20]:
#df = pd.DataFrame (list_name, columns = ['column_name'])

track_id = pd.DataFrame(track_ids, columns=['track_id'])

In [12]:
for track in track_ids:
    print(sp.audio_features(track)[0])

{'danceability': 0.591, 'energy': 0.764, 'key': 1, 'loudness': -5.484, 'mode': 1, 'speechiness': 0.0483, 'acousticness': 0.0383, 'instrumentalness': 0, 'liveness': 0.103, 'valence': 0.478, 'tempo': 169.928, 'type': 'audio_features', 'id': '5HCyWlXZPP0y6Gqq8TgA20', 'uri': 'spotify:track:5HCyWlXZPP0y6Gqq8TgA20', 'track_href': 'https://api.spotify.com/v1/tracks/5HCyWlXZPP0y6Gqq8TgA20', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5HCyWlXZPP0y6Gqq8TgA20', 'duration_ms': 141806, 'time_signature': 4}
{'danceability': 0.736, 'energy': 0.704, 'key': 3, 'loudness': -7.409, 'mode': 0, 'speechiness': 0.0615, 'acousticness': 0.0203, 'instrumentalness': 0, 'liveness': 0.0501, 'valence': 0.894, 'tempo': 149.995, 'type': 'audio_features', 'id': '27NovPIUIRrOZoCHxABJwK', 'uri': 'spotify:track:27NovPIUIRrOZoCHxABJwK', 'track_href': 'https://api.spotify.com/v1/tracks/27NovPIUIRrOZoCHxABJwK', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/27NovPIUIRrOZoCHxABJwK', 'duration_ms': 

{'danceability': 0.38, 'energy': 0.339, 'key': 3, 'loudness': -7.885, 'mode': 1, 'speechiness': 0.0338, 'acousticness': 0.691, 'instrumentalness': 0, 'liveness': 0.12, 'valence': 0.0849, 'tempo': 100.607, 'type': 'audio_features', 'id': '5CZ40GBx1sQ9agT82CLQCT', 'uri': 'spotify:track:5CZ40GBx1sQ9agT82CLQCT', 'track_href': 'https://api.spotify.com/v1/tracks/5CZ40GBx1sQ9agT82CLQCT', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5CZ40GBx1sQ9agT82CLQCT', 'duration_ms': 229227, 'time_signature': 4}
{'danceability': 0.748, 'energy': 0.74, 'key': 8, 'loudness': -6.01, 'mode': 1, 'speechiness': 0.0484, 'acousticness': 0.0107, 'instrumentalness': 2.18e-05, 'liveness': 0.101, 'valence': 0.518, 'tempo': 121.004, 'type': 'audio_features', 'id': '6OGogr19zPTM4BALXuMQpF', 'uri': 'spotify:track:6OGogr19zPTM4BALXuMQpF', 'track_href': 'https://api.spotify.com/v1/tracks/6OGogr19zPTM4BALXuMQpF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6OGogr19zPTM4BALXuMQpF', 'duration_ms'

{'danceability': 0.666, 'energy': 0.643, 'key': 8, 'loudness': -7.01, 'mode': 1, 'speechiness': 0.423, 'acousticness': 0.386, 'instrumentalness': 0, 'liveness': 0.283, 'valence': 0.756, 'tempo': 96.375, 'type': 'audio_features', 'id': '7wpyrkjEgz7W1vNYDxMKrF', 'uri': 'spotify:track:7wpyrkjEgz7W1vNYDxMKrF', 'track_href': 'https://api.spotify.com/v1/tracks/7wpyrkjEgz7W1vNYDxMKrF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7wpyrkjEgz7W1vNYDxMKrF', 'duration_ms': 105377, 'time_signature': 4}
{'danceability': 0.677, 'energy': 0.696, 'key': 0, 'loudness': -6.181, 'mode': 1, 'speechiness': 0.119, 'acousticness': 0.321, 'instrumentalness': 0, 'liveness': 0.42, 'valence': 0.464, 'tempo': 90.03, 'type': 'audio_features', 'id': '4iJyoBOLtHqaGxP12qzhQI', 'uri': 'spotify:track:4iJyoBOLtHqaGxP12qzhQI', 'track_href': 'https://api.spotify.com/v1/tracks/4iJyoBOLtHqaGxP12qzhQI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4iJyoBOLtHqaGxP12qzhQI', 'duration_ms': 198082, 't

{'danceability': 0.403, 'energy': 0.711, 'key': 8, 'loudness': -4.189, 'mode': 1, 'speechiness': 0.0569, 'acousticness': 0.137, 'instrumentalness': 0, 'liveness': 0.109, 'valence': 0.801, 'tempo': 203.803, 'type': 'audio_features', 'id': '7DhvNzyMM8jJWcfykG3psB', 'uri': 'spotify:track:7DhvNzyMM8jJWcfykG3psB', 'track_href': 'https://api.spotify.com/v1/tracks/7DhvNzyMM8jJWcfykG3psB', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7DhvNzyMM8jJWcfykG3psB', 'duration_ms': 170560, 'time_signature': 4}
{'danceability': 0.759, 'energy': 0.459, 'key': 8, 'loudness': -5.187, 'mode': 1, 'speechiness': 0.0948, 'acousticness': 0.00323, 'instrumentalness': 0, 'liveness': 0.0906, 'valence': 0.695, 'tempo': 109.997, 'type': 'audio_features', 'id': '2bgTY4UwhfBYhGT4HUYStN', 'uri': 'spotify:track:2bgTY4UwhfBYhGT4HUYStN', 'track_href': 'https://api.spotify.com/v1/tracks/2bgTY4UwhfBYhGT4HUYStN', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bgTY4UwhfBYhGT4HUYStN', 'duration_ms': 

{'danceability': 0.509, 'energy': 0.842, 'key': 6, 'loudness': -5.394, 'mode': 1, 'speechiness': 0.0512, 'acousticness': 0.0506, 'instrumentalness': 0, 'liveness': 0.0856, 'valence': 0.664, 'tempo': 171.901, 'type': 'audio_features', 'id': '2VmccNZz4zFmlcXstY8a2D', 'uri': 'spotify:track:2VmccNZz4zFmlcXstY8a2D', 'track_href': 'https://api.spotify.com/v1/tracks/2VmccNZz4zFmlcXstY8a2D', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2VmccNZz4zFmlcXstY8a2D', 'duration_ms': 190452, 'time_signature': 4}
{'danceability': 0.664, 'energy': 0.692, 'key': 1, 'loudness': -4.128, 'mode': 0, 'speechiness': 0.127, 'acousticness': 0.237, 'instrumentalness': 0, 'liveness': 0.0798, 'valence': 0.233, 'tempo': 154.924, 'type': 'audio_features', 'id': '5W8HXMOMLtXLz0RGKUtnlZ', 'uri': 'spotify:track:5W8HXMOMLtXLz0RGKUtnlZ', 'track_href': 'https://api.spotify.com/v1/tracks/5W8HXMOMLtXLz0RGKUtnlZ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5W8HXMOMLtXLz0RGKUtnlZ', 'duration_ms': 2

{'danceability': 0.462, 'energy': 0.615, 'key': 1, 'loudness': -6.429, 'mode': 1, 'speechiness': 0.215, 'acousticness': 0.23, 'instrumentalness': 0, 'liveness': 0.0928, 'valence': 0.307, 'tempo': 90.068, 'type': 'audio_features', 'id': '4YFPUYwA9Jmk9xQYCbz3zb', 'uri': 'spotify:track:4YFPUYwA9Jmk9xQYCbz3zb', 'track_href': 'https://api.spotify.com/v1/tracks/4YFPUYwA9Jmk9xQYCbz3zb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4YFPUYwA9Jmk9xQYCbz3zb', 'duration_ms': 182251, 'time_signature': 4}
{'danceability': 0.405, 'energy': 0.565, 'key': 1, 'loudness': -9.266, 'mode': 0, 'speechiness': 0.0372, 'acousticness': 0.38, 'instrumentalness': 0.0195, 'liveness': 0.107, 'valence': 0.166, 'tempo': 159.892, 'type': 'audio_features', 'id': '6G2xXQRSZlEzYsUEb3uvuR', 'uri': 'spotify:track:6G2xXQRSZlEzYsUEb3uvuR', 'track_href': 'https://api.spotify.com/v1/tracks/6G2xXQRSZlEzYsUEb3uvuR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6G2xXQRSZlEzYsUEb3uvuR', 'duration_ms': 2

In [16]:
features = sp.audio_features(track_ids)

In [28]:
features_df = pd.DataFrame(data=features, columns=features[0].keys())
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      100 non-null    float64
 1   energy            100 non-null    float64
 2   key               100 non-null    int64  
 3   loudness          100 non-null    float64
 4   mode              100 non-null    int64  
 5   speechiness       100 non-null    float64
 6   acousticness      100 non-null    float64
 7   instrumentalness  100 non-null    float64
 8   liveness          100 non-null    float64
 9   valence           100 non-null    float64
 10  tempo             100 non-null    float64
 11  type              100 non-null    object 
 12  id                100 non-null    object 
 13  uri               100 non-null    object 
 14  track_href        100 non-null    object 
 15  analysis_url      100 non-null    object 
 16  duration_ms       100 non-null    int64  
 17

In [30]:
df= features_df[['id', 'uri', 'track_href', 'analysis_url','type',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'speechiness','acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

In [31]:
df.head()

,id,uri,track_href,analysis_url,type,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,audio_features,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,141806,4
1,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,audio_features,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,212000,4
2,3ZozFqJJlDBNIm4xqxn2ZD,spotify:track:3ZozFqJJlDBNIm4xqxn2ZD,https://api.spotify.com/v1/tracks/3ZozFqJJlDBN...,https://api.spotify.com/v1/audio-analysis/3Zoz...,audio_features,0.649,0.760,1,-6.487,1,0.0574,0.114000,0.000000,0.3170,0.843,80.018,161853,4
3,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,audio_features,0.803,0.597,11,-6.035,0,0.1410,0.000619,0.000005,0.3230,0.331,136.008,257605,4
4,6PQ88X9TkUIAUIZJHW2upE,spotify:track:6PQ88X9TkUIAUIZJHW2upE,https://api.spotify.com/v1/tracks/6PQ88X9TkUIA...,https://api.spotify.com/v1/audio-analysis/6PQ8...,audio_features,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,231041,4


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                100 non-null    object 
 1   uri               100 non-null    object 
 2   track_href        100 non-null    object 
 3   analysis_url      100 non-null    object 
 4   type              100 non-null    object 
 5   danceability      100 non-null    float64
 6   energy            100 non-null    float64
 7   key               100 non-null    int64  
 8   loudness          100 non-null    float64
 9   mode              100 non-null    int64  
 10  speechiness       100 non-null    float64
 11  acousticness      100 non-null    float64
 12  instrumentalness  100 non-null    float64
 13  liveness          100 non-null    float64
 14  valence           100 non-null    float64
 15  tempo             100 non-null    float64
 16  duration_ms       100 non-null    int64  
 17

In [33]:
df_track= pd.concat([billboard,df], axis=1)

df_track.head()

,song,artist,last_week_rank,peak_rank,weeks_on_chart,id,uri,track_href,analysis_url,type,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Stay,The Kid LAROI & Justin Bieber,2,1,13,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,audio_features,...,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,141806,4
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,audio_features,...,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,212000,4
2,Fancy Like,Walker Hayes,5,3,16,3ZozFqJJlDBNIm4xqxn2ZD,spotify:track:3ZozFqJJlDBNIm4xqxn2ZD,https://api.spotify.com/v1/tracks/3ZozFqJJlDBN...,https://api.spotify.com/v1/audio-analysis/3Zoz...,audio_features,...,-6.487,1,0.0574,0.114000,0.000000,0.3170,0.843,80.018,161853,4
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,audio_features,...,-6.035,0,0.1410,0.000619,0.000005,0.3230,0.331,136.008,257605,4
4,Bad Habits,Ed Sheeran,6,2,15,6PQ88X9TkUIAUIZJHW2upE,spotify:track:6PQ88X9TkUIAUIZJHW2upE,https://api.spotify.com/v1/tracks/6PQ88X9TkUIA...,https://api.spotify.com/v1/audio-analysis/6PQ8...,audio_features,...,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,231041,4


In [34]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              100 non-null    object 
 1   artist            100 non-null    object 
 2   last_week_rank    100 non-null    object 
 3   peak_rank         100 non-null    object 
 4   weeks_on_chart    100 non-null    object 
 5   id                100 non-null    object 
 6   uri               100 non-null    object 
 7   track_href        100 non-null    object 
 8   analysis_url      100 non-null    object 
 9   type              100 non-null    object 
 10  danceability      100 non-null    float64
 11  energy            100 non-null    float64
 12  key               100 non-null    int64  
 13  loudness          100 non-null    float64
 14  mode              100 non-null    int64  
 15  speechiness       100 non-null    float64
 16  acousticness      100 non-null    float64
 17

In [37]:
X = df_track.select_dtypes(np.number)

from sklearn.preprocessing import StandardScaler

# K-Means is a distance based algorithm: we need to scale / normalize:

X_scaled = StandardScaler().fit_transform(X)

pd.DataFrame(X_scaled).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.519296,0.773731,-1.187630,0.304893,0.639101,-0.677336,-0.822611,-0.107332,-0.549124,-0.158619,1.461718,-1.402950,0.072954
1,0.563193,0.346492,-0.612507,-0.649509,-1.564697,-0.573287,-0.907266,-0.107332,-0.951582,1.681002,0.844217,0.213441,0.072954
2,-0.086301,0.745249,-1.187630,-0.192388,0.639101,-0.605606,-0.466589,-0.107332,1.078969,1.455472,-1.323589,-0.941318,0.072954
3,1.063378,-0.415420,1.687988,0.031711,-1.564697,0.053372,-0.999827,-0.107236,1.124617,-0.808677,0.410916,1.263609,0.072954
4,1.100705,1.720780,1.687988,1.183438,-1.564697,-0.783750,-0.782165,-0.106661,1.436541,0.341086,0.101685,0.651907,0.072954


In [39]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)


KMeans(n_clusters=4, random_state=42)

In [40]:
clusters = kmeans.predict(X_scaled)
df_track['cluster']=clusters

In [41]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              100 non-null    object 
 1   artist            100 non-null    object 
 2   last_week_rank    100 non-null    object 
 3   peak_rank         100 non-null    object 
 4   weeks_on_chart    100 non-null    object 
 5   id                100 non-null    object 
 6   uri               100 non-null    object 
 7   track_href        100 non-null    object 
 8   analysis_url      100 non-null    object 
 9   type              100 non-null    object 
 10  danceability      100 non-null    float64
 11  energy            100 non-null    float64
 12  key               100 non-null    int64  
 13  loudness          100 non-null    float64
 14  mode              100 non-null    int64  
 15  speechiness       100 non-null    float64
 16  acousticness      100 non-null    float64
 17

In [82]:
df_track

,song,artist,last_week_rank,peak_rank,weeks_on_chart,id,uri,track_href,analysis_url,type,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,Stay,The Kid LAROI & Justin Bieber,2,1,13,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,audio_features,...,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,141806,4,1
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,audio_features,...,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,212000,4,0
2,Fancy Like,Walker Hayes,5,3,16,3ZozFqJJlDBNIm4xqxn2ZD,spotify:track:3ZozFqJJlDBNIm4xqxn2ZD,https://api.spotify.com/v1/tracks/3ZozFqJJlDBN...,https://api.spotify.com/v1/audio-analysis/3Zoz...,audio_features,...,1,0.0574,0.114000,0.000000,0.3170,0.843,80.018,161853,4,1
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,audio_features,...,0,0.1410,0.000619,0.000005,0.3230,0.331,136.008,257605,4,0
4,Bad Habits,Ed Sheeran,6,2,15,6PQ88X9TkUIAUIZJHW2upE,spotify:track:6PQ88X9TkUIAUIZJHW2upE,https://api.spotify.com/v1/tracks/6PQ88X9TkUIA...,https://api.spotify.com/v1/audio-analysis/6PQ8...,audio_features,...,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,231041,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Esta Danada,Ivan Cornejo,-,96,1,6G2xXQRSZlEzYsUEb3uvuR,spotify:track:6G2xXQRSZlEzYsUEb3uvuR,https://api.spotify.com/v1/tracks/6G2xXQRSZlEz...,https://api.spotify.com/v1/audio-analysis/6G2x...,audio_features,...,0,0.0372,0.380000,0.019500,0.1070,0.166,159.892,214149,3,2
96,Whiskey And Rain,Michael Ray,-,97,1,1tf4Bfk8JIKryLpWYRvyiI,spotify:track:1tf4Bfk8JIKryLpWYRvyiI,https://api.spotify.com/v1/tracks/1tf4Bfk8JIKr...,https://api.spotify.com/v1/audio-analysis/1tf4...,audio_features,...,1,0.0244,0.508000,0.000003,0.2850,0.718,100.021,206507,4,0
97,TSU,Drake,84,9,5,4s7QLoImIwmPi9L6dq1nVW,spotify:track:4s7QLoImIwmPi9L6dq1nVW,https://api.spotify.com/v1/tracks/4s7QLoImIwmP...,https://api.spotify.com/v1/audio-analysis/4s7Q...,audio_features,...,1,0.4200,0.228000,0.000003,0.3900,0.246,130.019,308756,4,2
98,Love All,Drake Featuring JAY-Z,87,10,5,4VCbgIdr8ptegWeJpqLVHH,spotify:track:4VCbgIdr8ptegWeJpqLVHH,https://api.spotify.com/v1/tracks/4VCbgIdr8pte...,https://api.spotify.com/v1/audio-analysis/4VCb...,audio_features,...,1,0.2870,0.354000,0.000036,0.1500,0.155,92.131,228461,4,2


In [76]:
user_song = input("What song do you like? ")

What song do you like? Stay


In [77]:
print(df_track[df_track['song']==user_song]['cluster'].values[0])

1


In [112]:
c_group = df_track[df_track['song']==user_song]['cluster'].values[0]


#print(df_track[df_track['cluster']==c_group]['song'].values[0])

print(df_track[df_track['cluster']==c_group]['song'].sample(n=3, random_state=1))



58              Too Easy
72                 Volvi
35    Intro (Hate On Me)
Name: song, dtype: object


In [120]:
def song_choice():
    
    user_song = input("What song do you like? ")
    print("The cluster group is ", df_track[df_track['song']==user_song]['cluster'].values[0],".")
    
    c_group = df_track[df_track['song']==user_song]['cluster'].values[0] 
    print( df_track[df_track['cluster']==c_group]['song'].sample(n=3, random_state=1))
    #print("Here you have another song from the same cluster group :", df_track[df_track['cluster']==c_group]['song'].values[0],".")
    

In [121]:
song_choice()

What song do you like? Stay
The cluster group is  1 .
58              Too Easy
72                 Volvi
35    Intro (Hate On Me)
Name: song, dtype: object
